In [2]:
!pip install langchain_community
!pip install langchain


In [16]:
from langchain_mistralai import ChatMistralAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
load_dotenv()

True

In [25]:
## HELPERS

def get_prompt_txt(prompt_path) -> str:
    prompt_txt = ""
    with open(
        prompt_path, "r"
    ) as f:
        prompt_txt = f.read()  # has

    
    return prompt_txt



In [27]:
model = ChatMistralAI(model="open-mixtral-8x7b", temperature=0.7)
output_parser = StrOutputParser()


# QUERY Optimization
query_optimization_prompt = get_prompt_txt("../prompts/query_optimization.txt")

optimization_prompt_template =  ChatPromptTemplate.from_messages(
            [("human", query_optimization_prompt)],
        )
optimization_chain = optimization_prompt_template | model | output_parser


# Summarization

res_summary_prompt = get_prompt_txt("../prompts/result_summarization.txt")

res_summary_prompt_template =  ChatPromptTemplate.from_messages(
            [("human", res_summary_prompt)],
        )
summarization_chain = res_summary_prompt_template | model | output_parser




In [28]:
#1. optimize
res = optimization_chain.invoke({"q": "climate change"})

#2. call mosai with res


#3. get MOSAIC REsults and feed it to summarization_chain




Improving the climate change situation is a complex issue that requires the collective efforts of individuals, businesses, and governments around the world. Here are some ways to help improve the climate change situation:

1. Reduce your carbon footprint: You can reduce your carbon footprint by using less energy, driving less, and choosing low-carbon transportation options such as walking, biking, or taking public transportation. You can also reduce your carbon footprint by eating less meat, choosing locally grown food, and reducing waste.
2. Support renewable energy: You can support renewable energy by installing solar panels on your home, using wind power, or purchasing renewable energy credits. You can also support policies and politicians that promote renewable energy.
3. Advocate for climate change policies: You can advocate for climate change policies by contacting your elected officials, signing petitions, and participating in rallies and demonstrations. You can also join or don

Sure, I'd be happy to help you improve your query about climate change! However, I'll need a bit more information about what you're looking for. Here are a few questions that might help us get started:

* What is the specific topic or question you have about climate change?
* Are you looking for information from scientific studies, news articles, opinions, or something else?
* Do you have a preference for sources that are peer-reviewed, popular, or from a particular perspective (e.g. political, scientific, ethical)?
* Are you interested in a particular aspect of climate change, such as its causes, impacts, or solutions?

Once I have a better understanding of what you're looking for, I can help you craft a more specific and effective query. Here are a few general tips for improving your query:

* Use specific keywords related to your topic, such as "greenhouse gases," "sea level rise," or "renewable energy."
* Use quotation marks to search for exact phrases, such as "climate change deni